In [1]:
import random

import numpy as np
import scipy as sp

import scipy.linalg
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

In [2]:
from markovc import *

In [3]:
from data import *

In [4]:
alpha = 0.3
x = [alpha, 1-alpha, alpha*0.5]
Q = create_rate_matrix_from_vector(x)

P = embedded_jump_chain(Q)
pi = equi_dtmc(P)
P

array([[ 0.        ,  0.3       ,  0.7       ],
       [ 0.82352941,  0.        ,  0.17647059],
       [ 0.26086957,  0.73913043,  0.        ]])

In [5]:
equi_deriv_ctmc(Q, P, pi, (0,1))

array([ 0.39407949, -0.56080543,  0.16672594])

In [6]:
def nll(xhat, data):
    Qhat = create_rate_matrix_from_vector(xhat)
    
    s = 0
    for (winner, choices), count in data.items():
#         print("{} {} {}".format(winner, choices, count))
        winner_idx = [idx for idx, val in enumerate(choices) if winner==val][0]
        Qs = submatrix(Qhat, choices)
        try:
            piS = equi_ctmc(Qs)[winner_idx]
        except ValueError:
            print(xhat)
            raise
        
#         print(piS)

        s += count * np.log(np.abs(piS))    
        
    return -s

In [7]:
data = gen_data(Q, 5000)
x0 = [random.random() for _ in range(len(x))]

res = sp.optimize.minimize(lambda x: nll(x, data), x0, method="L-BFGS-B", bounds=[(0.0001, 0.9999) for _ in range(len(x0))])
print(res)

  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  success: True
        x: array([ 0.28202449,  0.70923999,  0.13709332])
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
     nfev: 48
      nit: 10
      fun: 3645.2590380467964
   status: 0
      jac: array([-0.00040927, -0.00077307, -0.00013642])


In [8]:
xopt = res['x']
Qhat = create_rate_matrix_from_vector(xopt)

In [9]:
Qhat

array([[-0.99126448,  0.28202449,  0.70923999],
       [ 0.71797551, -0.85506883,  0.13709332],
       [ 0.29076001,  0.86290668, -1.15366669]])

In [10]:
Phat = embedded_jump_chain(Qhat)
pihat = equi_ctmc(Qhat)

In [11]:
equi_deriv_ctmc(Qhat, Phat, pihat, (2, 1))

array([-0.11654506, -0.32660184,  0.4431469 ])

In [12]:
nll(xopt, data)

3645.2590380467964